<a href="https://colab.research.google.com/github/CosmoStat/ShearDataClub/blob/main/KiDS1000/DataExploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data exploration of the KiDS-1000 gold shear catalog

In this notebook, we are using the data provided here: http://kids.strw.leidenuniv.nl/DR4/KiDS-1000_shearcatalogue.php

In [ ]:
# Retrieve the data from fast storage on Google cloud, takes 2mins
!wget https://storage.googleapis.com/ahw2019/KiDS_DR4.1_ugriZYJHKs_SOM_gold_WL_cat.fits

In [ ]:
from astropy.table import Table
# Loading the data
cat = Table.read('KiDS_DR4.1_ugriZYJHKs_SOM_gold_WL_cat.fits')